https://github.com/Toblerity/Fiona/issues/944

In [ ]:
import fiona

In [ ]:
import maup
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.updaters import Tally, cut_edges, exterior_boundaries, exterior_boundaries_as_a_set
from networkx import is_connected, connected_components


In [ ]:
graph = Graph.from_file("./TX_vtds.zip", ignore_errors=True)
precincts = geopandas.read_file("./TX_vtds.zip")

In [ ]:
components = list(connected_components(graph))
biggest_component_size = max(len(c) for c in components)
problem_components = [c for c in components if len(c) != biggest_component_size]
for component in problem_components:
    for node in component:
        graph.remove_node(node)
is_connected(graph)

In [ ]:
election = Election("PRES16", {"Dem": "PRES16D", "Rep": "PRES16R"})

initial_partition = GeographicPartition(
    graph,
    assignment="USCD",
    updaters={
        "cut_edges": cut_edges,
        "population": Tally("TOTPOP", alias="population"),
        "PRES16": election
    }
)

In [ ]:
initial_partition.graph

In [ ]:
initial_partition.assignment.keys

In [ ]:
def convexhull

In [ ]:
print(initial_partition.graph.geometry.get(8936))

In [ ]:
for district, pop in initial_partition["population"].items():
    print("District {}: {}".format(district, pop))

In [ ]:
from gerrychain.constraints.contiguity import contiguous_components, contiguous
from gerrychain import Partition
contiguous_components(initial_partition)

In [ ]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas

In [ ]:
sum_population = sum(initial_partition["population"].values())
ideal_population = sum_population / len(initial_partition)

# We use functools.partial to bind the extra parameters (pop_col, pop_target, epsilon, node_repeats)
# of the recom proposal.
proposal = partial(recom,
                   pop_col="TOTPOP",
                   pop_target=sum_population,
                   epsilon=1,
                   node_repeats=2
                  )

In [ ]:
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(initial_partition["cut_edges"])
)

pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, 1)

In [ ]:
from gerrychain import MarkovChain
from gerrychain.constraints import single_flip_contiguous, contiguous
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept
steps = 100
chain = MarkovChain(
    proposal=proposal,
    constraints=[single_flip_contiguous],
    accept=always_accept,
    initial_state=initial_partition,
    total_steps=steps
)

In [ ]:
last1 = None
for partition in chain.with_progress_bar():
    last1 = partition

In [ ]:
last1.plot(figsize=(10, 10), cmap="tab20")
plt.axis('off')
plt.show()

In [ ]:
last1

In [ ]:
initial_partition = GeographicPartition(
    graph,
    assignment="USCD",
    updaters={
        "cut_edges": cut_edges,
        "population": Tally("TOTPOP", alias="population"),
        "PRES16": election
    }
)
initial_partition.assignment = last1.assignment
initial_partition.subgraphs = last1.subgraphs

In [ ]:
steps = 100000
chain = MarkovChain(
    proposal=propose_random_flip,
    constraints=[single_flip_contiguous, pop_constraint, compactness_bound],
    accept=always_accept,
    initial_state=initial_partition,
    total_steps=steps
)

In [ ]:
last2 = None
for partition in chain.with_progress_bar():
    last2 = partition

In [ ]:
last2.plot(figsize=(10, 10), cmap="tab20")
plt.axis('off')
plt.show()